In [53]:
from google.colab import drive
drive.mount('/content/drive')

from google.colab import files
import io




#Começando por importar as bibliotecas basicas numpy e pandas
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

#credit_data é o csv de estudo, esse data set foi retirado do site 'UCI - Machine Learning Repository', neste data set temos
#dados relativos a pessoas e seus ganhos anuais e outros dados

#Os dados estão sem inconsistências e possuem um "?" quando o dado não esta presente, sendo o "?" utilizado por alguns 
#programas e bibliotecas para identificar a ausência da informação. 

base = pd.read_csv(io.BytesIO(uploaded['base_census.csv']))

#base = pd.read_csv('https://drive.google.com/drive/folders/1duOJm7CO3zhh0Kw55D-QTGrdGtE6746s/base_census.csv');

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [54]:
base
#o objetivo será encontrar resultados pelo treinamento baseado na coluna income, que será nossa base de resposta

,age,workclass,final-weight,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loos,hour-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [55]:
base2 = base.drop(['income'], axis=1)

base2

#confira que as colunas, workclass, education, marital-status, occupation, relationship, race, sex e native-country são do tipo 
#categóricas, portanto será necessário codificar essas colunas para torná-las númericas

# 

,age,workclass,final-weight,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loos,hour-per-week,native-country
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States


In [56]:
base_race_dummies = pd.get_dummies(base2['race'])
base2 = pd.concat([base2, base_race_dummies], axis=1)

previsores = base2.iloc[:, 0: np.size(base2,1)].values
previsores

array([[39, ' State-gov', 77516, ..., 0, 0, 1],
       [50, ' Self-emp-not-inc', 83311, ..., 0, 0, 1],
       [38, ' Private', 215646, ..., 0, 0, 1],
       ...,
       [58, ' Private', 151910, ..., 0, 0, 1],
       [22, ' Private', 201490, ..., 0, 0, 1],
       [52, ' Self-emp-inc', 287927, ..., 0, 0, 1]], dtype=object)

In [57]:
#pegando apenas os registros de pagou ou não como referênca de classes
classe = base.iloc[:, 14].values
classe

array([' <=50K', ' <=50K', ' <=50K', ..., ' <=50K', ' <=50K', ' >50K'],
      dtype=object)

In [58]:
#Se observarmos, muitos dados são do tipo categoricos, então precisaremos converter estes para númericos seguindo os passos
#a seguir.

from sklearn.preprocessing import LabelEncoder,OneHotEncoder

labelencoder_previsores =  LabelEncoder()
#transformando os registros categoricos em númericos, aqui no caso, apenas nas colunas que são necessárias as transformações


previsores[:,1]=labelencoder_previsores.fit_transform(previsores[:,1])
previsores[:,3]=labelencoder_previsores.fit_transform(previsores[:,3])
previsores[:,5]=labelencoder_previsores.fit_transform(previsores[:,5])
previsores[:,6]=labelencoder_previsores.fit_transform(previsores[:,6])
previsores[:,7]=labelencoder_previsores.fit_transform(previsores[:,7])
previsores[:,8]=labelencoder_previsores.fit_transform(previsores[:,8])
previsores[:,9]=labelencoder_previsores.fit_transform(previsores[:,9])
previsores[:,13]=labelencoder_previsores.fit_transform(previsores[:,13])

previsores


array([[39, 7, 77516, ..., 0, 0, 1],
       [50, 6, 83311, ..., 0, 0, 1],
       [38, 4, 215646, ..., 0, 0, 1],
       ...,
       [58, 4, 151910, ..., 0, 0, 1],
       [22, 4, 201490, ..., 0, 0, 1],
       [52, 5, 287927, ..., 0, 0, 1]], dtype=object)

In [59]:

labelencoder_classe =  LabelEncoder()
classe = labelencoder_classe.fit_transform(classe)
classe

array([0, 0, 0, ..., 0, 0, 1])

In [60]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
previsores = scaler.fit_transform(previsores)
previsores

array([[ 0.03067056,  2.15057856, -1.06361075, ..., -0.32576824,
        -0.09161163,  0.4130197 ],
       [ 0.83710898,  1.46373585, -1.008707  , ..., -0.32576824,
        -0.09161163,  0.4130197 ],
       [-0.04264203,  0.09005041,  0.2450785 , ..., -0.32576824,
        -0.09161163,  0.4130197 ],
       ...,
       [ 1.42360965,  0.09005041, -0.35877741, ..., -0.32576824,
        -0.09161163,  0.4130197 ],
       [-1.21564337,  0.09005041,  0.11095988, ..., -0.32576824,
        -0.09161163,  0.4130197 ],
       [ 0.98373415,  0.77689313,  0.92989258, ..., -0.32576824,
        -0.09161163,  0.4130197 ]])

In [0]:
from sklearn.model_selection import train_test_split

previsores_treinamento, previsores_teste, classe_treinamento, classe_teste = train_test_split(previsores, classe, test_size = 0.75, random_state=0) 

In [62]:
print(previsores_treinamento.size)
print(previsores_teste.size)
print(classe_treinamento.size)
print(classe_teste.size)

154660
463999
8140
24421


In [63]:
#Naive Bayes
#Naive Bayes
#Naive Bayes
#Naive Bayes


#Primeira prática de machine learning a ser utilizado será o naive bayes
from sklearn.naive_bayes import GaussianNB
classificador = GaussianNB()
classificador.fit(previsores_treinamento, classe_treinamento)

GaussianNB(priors=None, var_smoothing=1e-09)

In [64]:
previsoes = classificador.predict(previsores_teste)
previsoes

array([0, 0, 0, ..., 0, 0, 1])

In [65]:
from sklearn.metrics import confusion_matrix, accuracy_score
precisao = accuracy_score(classe_teste, previsoes)
matriz = confusion_matrix(classe_teste, previsoes)
print(precisao) #percentual de acerto
print(matriz) #matriz de confusão

0.8204004750010238
[[16452  2038]
 [ 2348  3583]]


In [66]:
#Arvore de decisão
#Arvore de decisão
#Arvore de decisão
#Arvore de decisão

from sklearn.tree import DecisionTreeClassifier
classificador = DecisionTreeClassifier(criterion='entropy', random_state=0)
classificador.fit(previsores_treinamento, classe_treinamento)
previsoes = classificador.predict(previsores_teste)

from sklearn.metrics import confusion_matrix, accuracy_score
precisao = accuracy_score(classe_teste, previsoes)
matriz = confusion_matrix(classe_teste, previsoes)
print(precisao) #percentual de acerto
print(matriz) #matriz de confusão

0.8086482945006347
[[16215  2275]
 [ 2398  3533]]


In [67]:
#Random Forest
#Random Forest
#Random Forest
#Random Forest
from sklearn.ensemble import RandomForestClassifier
classificador = RandomForestClassifier(n_estimators=40, criterion='entropy', random_state=0)
classificador.fit(previsores_treinamento, classe_treinamento)
previsoes = classificador.predict(previsores_teste)

from sklearn.metrics import confusion_matrix, accuracy_score
precisao = accuracy_score(classe_teste, previsoes)
matriz = confusion_matrix(classe_teste, previsoes)
print(precisao) #percentual de acerto
print(matriz) #matriz de confusão

0.8532410630195324
[[17319  1171]
 [ 2413  3518]]


In [68]:

#KNN
#KNN
#KNN
#KNN




from sklearn.model_selection import train_test_split
previsores_treinamento, previsores_teste, classe_treinamento, classe_teste = train_test_split(previsores, classe, test_size=0.25, random_state=0)

from sklearn.neighbors import KNeighborsClassifier
classificador = KNeighborsClassifier(n_neighbors=5, metric='minkowski', p = 2)
classificador.fit(previsores_treinamento, classe_treinamento)
previsoes = classificador.predict(previsores_teste)

from sklearn.metrics import confusion_matrix, accuracy_score
precisao = accuracy_score(classe_teste, previsoes)
matriz = confusion_matrix(classe_teste, previsoes)
print(precisao)
print(matriz)
import collections
collections.Counter(classe_teste)

0.8243459034516644
[[5567  592]
 [ 838 1144]]


Counter({0: 6159, 1: 1982})

In [69]:
#Regressão Logística
#Regressão Logística
#Regressão Logística
#Regressão Logística
from sklearn.linear_model import LogisticRegression
classificador = LogisticRegression(random_state = 1)
classificador.fit(previsores_treinamento, classe_treinamento)
previsoes = classificador.predict(previsores_teste)

from sklearn.metrics import confusion_matrix, accuracy_score
precisao = accuracy_score(classe_teste, previsoes)
matriz = confusion_matrix(classe_teste, previsoes)
print(precisao)
print(matriz)

import collections
collections.Counter(classe_teste)


0.8217663677680874
[[5781  378]
 [1073  909]]


Counter({0: 6159, 1: 1982})

In [70]:
#SVM
#SVM
#SVM
#SVM

from sklearn.svm import SVC
classificador = SVC(kernel = 'rbf', random_state = 1, C = 2.0)
classificador.fit(previsores_treinamento, classe_treinamento)
previsoes = classificador.predict(previsores_teste)

from sklearn.metrics import confusion_matrix, accuracy_score
precisao = accuracy_score(classe_teste, previsoes)
matriz = confusion_matrix(classe_teste, previsoes)
print(precisao)
print(matriz)

import collections
collections.Counter(classe_teste)

0.8452278589853827
[[5796  363]
 [ 897 1085]]


Counter({0: 6159, 1: 1982})

In [71]:
#Redes Neurais
#Redes Neurais
#Redes Neurais
#Redes Neurais

from sklearn.neural_network import MLPClassifier
classificador = MLPClassifier(verbose = True,
                              max_iter=1000,
                              tol = 0.0000010,
                              solver = 'adam',
                              hidden_layer_sizes=(100),
                              activation='relu')
classificador.fit(previsores_treinamento, classe_treinamento)
previsoes = classificador.predict(previsores_teste)

from sklearn.metrics import confusion_matrix, accuracy_score
precisao = accuracy_score(classe_teste, previsoes)
matriz = confusion_matrix(classe_teste, previsoes)

Iteration 1, loss = 0.47496758
Iteration 2, loss = 0.37154663
Iteration 3, loss = 0.34900921
Iteration 4, loss = 0.33641317
Iteration 5, loss = 0.32955640
Iteration 6, loss = 0.32637728
Iteration 7, loss = 0.32484883
Iteration 8, loss = 0.32369952
Iteration 9, loss = 0.32234672
Iteration 10, loss = 0.32140665
Iteration 11, loss = 0.32089933
Iteration 12, loss = 0.32041437
Iteration 13, loss = 0.31981338
Iteration 14, loss = 0.31887533
Iteration 15, loss = 0.31880383
Iteration 16, loss = 0.31779460
Iteration 17, loss = 0.31763460
Iteration 18, loss = 0.31708117
Iteration 19, loss = 0.31640851
Iteration 20, loss = 0.31649660
Iteration 21, loss = 0.31578145
Iteration 22, loss = 0.31561011
Iteration 23, loss = 0.31494802
Iteration 24, loss = 0.31445515
Iteration 25, loss = 0.31424724
Iteration 26, loss = 0.31422069
Iteration 27, loss = 0.31389735
Iteration 28, loss = 0.31366644
Iteration 29, loss = 0.31295951
Iteration 30, loss = 0.31319274
Iteration 31, loss = 0.31252787
Iteration 32, los

In [72]:
print(precisao)
print(matriz)

0.8405601277484338
[[5767  392]
 [ 906 1076]]


In [80]:
#Redes neurais com Keras
#Redes neurais com Keras
#Redes neurais com Keras
#Redes neurais com Keras




import tensorflow as tf
import keras

from keras.models import Sequential
from keras.layers import Dense





classificador = Sequential()
classificador.add(Dense(units = 2, activation = 'relu', input_dim = 19))
classificador.add(Dense(units = 2, activation = 'relu'))
classificador.add(Dense(units = 1, activation = 'sigmoid'))
classificador.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

classificador.fit(previsores_treinamento, classe_treinamento, batch_size = 10, nb_epoch = 100)
previsoes = classificador.predict(previsores_teste)
previsoes = (previsoes > 0.5)

from sklearn.metrics import confusion_matrix, accuracy_score
precisao = accuracy_score(classe_teste, previsoes)
matriz = confusion_matrix(classe_teste, previsoes)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/100
24420/24420 [==============================] - 3s 114us/step - loss: 0.5846 - accuracy: 0.7424
Epoch 2/100
24420/24420 [==============================] - 3s 106us/step - loss: 0.4146 - accuracy: 0.8271
Epoch 3/100
24420/24420 [==============================] - 3s 106us/step - loss: 0.3878 - accuracy: 0.8343
Epoch 4/100
24420/24420 [==============================] - 3s 107us/step - loss: 0.3828 - accuracy: 0.8340
Epoch 5/100
24420/24420 [==============================] - 3s 107us/step - loss: 0.3808 - accuracy: 0.8356
Epoch 6/100
24420/24420 [==============================] - 3s 109us/step - loss: 0.3801 - accuracy: 0.8360
Epoch 7/100
24420/24420 [==============================] - 3s 106us/step - loss: 0.3795 - accuracy: 0.8362
Epoch 8/100
24420/24420 [==============================] - 3s 107us/step - loss: 0.3793 - accuracy: 0.8359
Epoch 9/100
24420/24420 [==============================] - 3s 111us/step - loss: 0.3788 - accuracy: 0.8362
Epoch 10/100
24420/24420 [===========

In [77]:
print(precisao)
print(matriz)

0.831961675469844
[[5804  355]
 [1013  969]]


In [0]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [52]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

<IPython.core.display.Javascript object>

KeyboardInterrupt: ignored